pip install pymssql 



In [141]:
import pandas as pd
import pymssql 

In [142]:
server="SQL-retail2.nikamed.local"
base="Retail2_shops"

In [482]:
sql="""
with mag_iskl as (
SELECT CAST(_IDRRef AS uniqueidentifier) AS Ссылка, --исключение техн склада
_IDRRef, 
_Code AS Код
FROM _Reference10
where _Code='0-002'
),
--------------------------------------исх продажи-----------------------
sales_ish as (
select 
       cast(dateadd(year, -2000, d21._date_time) as date) as Дата , 
       cast(d21._number as varchar(20))+'_'+ cast(dateadd(year, -2000, d21._date_time) as varchar(50)) as  Num4ККМ,  
       iif(e._EnumOrder=1,-1,1) as 'ВидОперации'
       ,iif(e._EnumOrder=1, -d21t._Fld6384, d21t._Fld6384 ) as 'Количество'
       ,iif(e._EnumOrder=1, -d21t._Fld6397, d21t._Fld6397) as 'Сумма'
              , cast(r10._idrref as uniqueidentifier) as 'ccМагазин'
       , iif(r2_10._Code is null, cast(r10._idrref as uniqueidentifier), 
                                  cast(r2_10._idrref as uniqueidentifier) ) as 'ccМагазинМотивационный'
from _Document21_VT6378 d21t
left outer join _Document21 d21 on d21t._Document21_IDRRef = d21._IDRRef /*исх чек*/
LEFT OUTER JOIN _reference47 r47 ON d21._Fld6331RRef = r47._idrref /*вид документа*/
LEFT OUTER JOIN _Enum352 e ON d21._Fld6333RRef = e._idrref /*вид документа в перечислениях чека ккм - 0-продажа, 1-возврат*/
left outer join _Reference10 r10 on d21._Fld6339RRef = r10._idrref /*shop*/
left outer join  _Document21 d2_21 on d21._Fld6353RRef=d2_21._idrref /*поиск мотив маг*/
left outer join _Reference10 r2_10 on d2_21._Fld6339RRef = r2_10._idrref /*shop*/
where d21._marked = 0 and d21._Posted = 1
and d21._date_time >= '08.01.4021' --and d21._date_time < '09.01.4021'
union all
select 
       cast(dateadd(year, -2000, d._date_time) as date) as 'Дата', 
        cast(d._number as varchar(20))+'_'+ cast(dateadd(year, -2000, d._date_time) as varchar(50)) as  Num4ККМ,      
        -1
       , -dt._Fld3984 as 'Количество'
       , -dt._Fld3988 as 'СуммаЧека'
       , cast(r10._idrref as uniqueidentifier) as 'ccМагазин'
       , iif(r2_10._Code is null, cast(r10._idrref as uniqueidentifier), cast(r2_10._idrref as uniqueidentifier) ) as 'ccМагазинМотивационный'
from _Document216_VT3980 dt
left outer join _Document216 d on dt._Document216_IDRRef = d._IDRRef /*исх возвратный чек*/
left outer join _Reference10 r10 on d._Fld3950RRef = r10._idrref /*shop*/
left outer join  _Reference172 r172  on dt._Fld4001RRef = r172._idrref
left outer join  _Reference23 r23  on dt._Fld3982RRef = r23._idrref
left outer join _Reference131 r131 on d._Fld11137RRef= r131._idrref --  склады
left outer join _Reference10 r2_10 on r131._Fld2686RRef = r2_10._idrref /*shop к складу*/
where d._marked = 0 and d._Posted = 1
and d._date_time >= '08.01.4021' --and d._date_time  < '09.01.4021'

), 
-------------суммарные продажи------------------------------------ 
sales_sum2 as (

SELECT

Дата as date_ss, 
sum(Сумма)  as Summa, 
ccМагазин, 
ccМагазинМотивационный, 
ВидОперации,
Num4ККМ 

FROM sales_ish

group by Дата, ccМагазин, ccМагазинМотивационный, ВидОперации, Num4ККМ

),
-------------c кол чеков------------------------------------ 
sales_sum as (
SELECT date_ss,

ccМагазин, 
ccМагазинМотивационный,
sum(Summa) as Summa, 
sum(ВидОперации) as Kol4
FROM sales_sum2

group by date_ss, ccМагазин, ccМагазинМотивационный

),
-----------трафиk группировка-----------------------------
tr_gr as (
SELECT cast(dateadd(year,-2000,i._fld10076) AS date) AS date_ii,
		 sum(i._fld10077) AS Трафик,
		 cast( r._Fld2907RRef AS uniqueidentifier) AS cсМагазин
FROM _InfoRg10074 i 

left outer JOIN _reference142 r ON i._Fld10075RRef = r._IDRRef 
left outer JOIN mag_iskl mi ON r._Fld2907RRef=mi._IDRRef

WHERE i._fld10076 >= '08.01.4021'
--		AND i._fld10076 < '09.01.4021'
		AND i._fld10077 > 0
		AND mi._IDRRef is  null
GROUP BY  cast(dateadd(year, -2000, i._fld10076) AS date) , cast( r._Fld2907RRef AS uniqueidentifier)
),
---------------------------------------------------
--select * from tr_gr
traf_ich as (
SELECT tab.date_ii, 
    tab.Трафик, 
    tab.cсМагазин, 
    iif(sum(sales_sum.Summa) is null,0,sum(sales_sum.Summa)) as Summa_ii, 
    sum(iif(Kol4 is null,0,Kol4)) as Kol4
    
    
FROM tr_gr tab

left outer join sales_sum on sales_sum.date_ss=tab.date_ii and sales_sum.ccМагазин=tab.cсМагазин
--!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
WHERE tab.date_ii >= '08.01.2021' --and tab.date_ii < '09.01.2021'
--!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
group by tab.date_ii , tab.cсМагазин, tab.Трафик
),
------------------сборка факт показателей---------------
tab_fact as (
select tab.date_ii ,
    concat(left(tab.date_ii,8),'01') As period,
    tab.Трафик,
    tab.cсМагазин as ccShops,
    tab.Summa_ii,
    tab.Kol4,
    iif(tab.Kol4=0,0,tab.Summa_ii/tab.Kol4)  as sr4,
    iif(tab.Трафик=0,0,tab.Kol4/tab.Трафик)  as konv
    
    from traf_ich as tab
),    
--------------------сбока факта к плану--------------------------

--------------план конверсии---------------------
pl_konv as (
SELECT cast(dateadd(year,
		 -2000,
		 _Fld7062) AS date) AS period,
		 (d_v._Fld7072/100) AS ПланКонверсии,
		 cast(d_v._Fld7071RRef AS uniqueidentifier) AS ccShops
FROM _Document302_VT7069 d_v 

left outer JOIN _Document302 d ON d_v._Document302_idrref = d._idrref

WHERE d._Marked = 0
		AND d._Posted=1
--!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        and _Fld7062 = '4021-08-01'
--!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
)


------------------сборка----------------------

select 

    
    pk.period,
    pk.ПланКонверсии as PlanKonv,
    pk.ccShops,
    tf.Kol4 as Kol4,
    tf.Summa_ii,
    tf.sr4 as Avg4,
    tf.konv as konv
--    * 

from pl_konv pk    

LEFT outer join tab_fact tf ON tf.period=pk.period and tf.ccShops = pk.ccShops


"""

In [483]:
def read_sql(sql,base, serv):
    #with pymssql.connect(server=serv ,database = base ,charset =  'cp1251') as  conn:
    with pymssql.connect(server=serv ,charset =  'cp1251',database = base ) as  conn:                  
    
        cursor = conn.cursor()  
        df = pd.read_sql( sql,conn)
    return df

In [484]:
%%time
df_tr=read_sql(sql,base, server)

Wall time: 13.9 s


In [485]:
df_tr

,period,PlanKonv,ccShops,Kol4,Summa_ii,Avg4,konv
0,2021-08-01,0.18,15000f8d-015d-0443-11e9-91dc0bd5fdd6,7.0,43428.0,6204.000000,0.104478
1,2021-08-01,0.22,4a3cb3b9-7492-28df-11e7-155be2ea26ad,5.0,39753.0,7950.600000,0.078125
2,2021-08-01,0.28,4a3ce78c-7492-28df-11e6-f8d0f307b036,12.0,54285.0,4523.750000,0.315789
3,2021-08-01,0.30,4a3c3cb8-7492-28df-11e7-778fe6fa7541,8.0,45406.0,5675.750000,0.222222
4,2021-08-01,0.32,4a3c0d86-7492-28df-11e6-b2f653a1d1a8,14.0,104225.0,7444.642857,0.254545
...,...,...,...,...,...,...,...
5767,2021-08-01,0.23,4a3ccd94-7492-28df-11e6-ff1c89f945f0,14.0,69891.0,4992.214285,0.241379
5768,2021-08-01,0.26,4a3cac86-7492-28df-11e3-b3f310efba67,11.0,59895.0,5445.000000,0.164179
5769,2021-08-01,0.33,4a3c90b8-7492-28df-11e1-bc6a07dfd167,6.0,17845.0,2974.166666,0.171429
5770,2021-08-01,0.00,4a3caeb7-7492-28df-11e7-2a91578c98c1,NaN,NaN,NaN,NaN


In [470]:
df_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5772 entries, 0 to 5771
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   period         5772 non-null   object 
 1   ПланКонверсии  5772 non-null   float64
 2   ccShops        5772 non-null   object 
 3   Kol4           5747 non-null   float64
 4   Summa_ii       5747 non-null   float64
 5   Avg4           5747 non-null   float64
 6   konv           5747 non-null   float64
dtypes: float64(5), object(2)
memory usage: 315.8+ KB


df_tr['Summa_ii']=df_tr['Summa_ii'].fillna(0)
df_tr.info()

In [253]:
sum(df_tr['Summa_ii'])

531986938.41999996

In [254]:
sum(df_tr['Трафик'])


373766.0

In [255]:
sum(df_tr['Kol4'])

87299

In [256]:
df_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5902 entries, 0 to 5901
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date_ii    5902 non-null   object 
 1   Трафик     5902 non-null   float64
 2   cсМагазин  5902 non-null   object 
 3   Summa_ii   5902 non-null   float64
 4   Kol4       5902 non-null   int64  
 5   sr4        5902 non-null   float64
 6   konv       5902 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 322.9+ KB


In [320]:
sql="""
--------------план конверсии---------------------
SELECT cast(dateadd(year,
		 -2000,
		 _Fld7062) AS date) AS Период,
		 (d_v._Fld7072/100) AS Конверсия,
		 cast(d_v._Fld7071RRef AS uniqueidentifier) AS ссМагазин
FROM _Document302_VT7069 d_v left outer
JOIN _Document302 d
	ON d_v._Document302_idrref = d._idrref
WHERE d._Marked = 0
		AND d._Posted=1
--!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        and _Fld7062 = '4021-08-01'
--!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
"""

conn.close()

In [321]:
%%time
df=read_sql(sql,base, server)


Wall time: 56 ms


In [322]:
df.head()


,Период,Конверсия,ссМагазин
0,2021-08-01,0.0,4a3ccdba-7492-28df-11e4-34cd06df5674
1,2021-08-01,0.0,4a3cd4b8-7492-28df-11e5-decf120e76ce
2,2021-08-01,0.0,4a3cb8ae-7492-28df-11e3-63e1be18ea68
3,2021-08-01,0.0,4a3cb8ae-7492-28df-11e3-63e1be18ea7a
4,2021-08-01,0.0,4a3c90b8-7492-28df-11e1-8e081c40ef1a
